In [1]:
# Python code that matches customer names within a list
# need fuzzywuzzy installed
import pandas
import numpy
import sys

In [2]:
# basic input of csv and print column names
filename = '~/Documents/Projects/16_01_18_largeagribusiness/ParentInfo/rootMatching_oils_SAP.csv'
df = pandas.read_csv(filename, encoding='latin-1')
print(df.columns.values)

['Root Customer Name' 'Root Customer ID']


In [3]:
# This tile does the matching. The process works as follows.
# 1. Remove all the generic words that may match in several customers name such as 'company'
# 2. Iterate over the customer list and for each customer iterate over the same customer list.
# 3. Record the best matches based on a score produced by fuzzywuzzy and output it into a dataframe.
# The best matches fields are only filled if the match is higher than a certain score
from fuzzywuzzy import fuzz
# build matched dataframe
df_matched = pandas.DataFrame(columns=['Orig Cust Name','Orig Cust Name2','Customer Name',\
                                        'Customer Name2','Customer ID', 'Customer ID 2', 'Matched ID', 'Score'])
# to improve performance of match remove generic words and remove upper case
# generic words have to picked manually
df['Orig Cust Name'] = df['Root Customer Name']
df['Root Customer Name'] = df['Root Customer Name'].str.lower()
df['Root Customer Name'] = df['Root Customer Name'].str.replace('family trust', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('farms', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('farm', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('ltd', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('pty', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('australia', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('new zealand', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('limited', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('stockfeed', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('pastoral', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('company', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('partnership', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('enterprises', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('solutions', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('trust', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('holdings', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('dairies', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('australasia', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('rural', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('supplies', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('tavern', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('hotel', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('dairy', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('products', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('transport', '')
df['Root Customer Name'] = df['Root Customer Name'].str.replace('national', '')

print(df.head())
# create copy of the list for matching
df2 = df.copy()

# iterate over the list
matchingLimit = 77
bestScore = 0
for i, row in df.iterrows():
    # for each element in the list iterate over the copy to find matches
    for j, row2 in df2.iterrows():
        
        customerName = row['Root Customer Name']
        customerName2 = row2['Root Customer Name']
        # have to set the best match as the first match, making sure to avoid matching to itself
        if j==0 and fuzz.token_sort_ratio(row['Root Customer ID'], row2['Root Customer ID']) != 100:
            bestMatchedName = customerName2
            bestMatchedId = row2['Root Customer ID']
            bestScore = fuzz.token_sort_ratio(customerName2, customerName)
            bestOrigCust = row2['Orig Cust Name']
        # for the first customer in the list you need an alternative first match as the standard
        # first match is itself
        elif j==1 and fuzz.token_sort_ratio(row['Root Customer ID'], row2['Root Customer ID']) != 100:
            bestMatchedName = customerName2
            bestMatchedId = row2['Root Customer ID']
            bestScore = fuzz.token_sort_ratio(customerName2, customerName)   
            bestOrigCust = row2['Orig Cust Name']
        # update the match when the score is improved
        elif fuzz.token_sort_ratio(customerName2, customerName) > bestScore and \
             fuzz.token_sort_ratio(row['Root Customer ID'], row2['Root Customer ID']) !=100:
            bestScore = fuzz.token_sort_ratio(customerName2, customerName)
            bestMatchedName = customerName2
            bestMatchedId = row2['Root Customer ID']
            bestOrigCust = row2['Orig Cust Name']
            
    # This writes the match to the matching fields if the score is sufficient
    if bestScore > matchingLimit:
        matchedID = bestMatchedId
    else:
        matchedID = row['Root Customer ID']
    if i%100==0: print(i) # for progress update
    #if i>100: break # can be used in testing for just a few customer names

    # fill the matching dataframe
    df_matched.loc[i] = (row['Orig Cust Name'], bestOrigCust, customerName, bestMatchedName, \
                         row['Root Customer ID'], bestMatchedId, matchedID, bestScore)

    

df_matched.to_csv("TEST_customer_matching.csv")

# this outputs a csv file that you can then manually check. You can 'correct' matches by
# putting the Matched id column as the original Customer Id if it is an incorrect match.
# Do the same with the matched and customer names

            Root Customer Name  Root Customer ID  \
0             moore wilson co               1036   
1    baking technical advice              103788   
2             anchor manawatu             106269   
3  country foods gisborne 1997             10789   
4                   commercial                11   

                    Orig Cust Name  
0              MOORE WILSON CO LTD  
1  BAKING TECHNICAL ADVICE PTY LTD  
2          ANCHOR MANAWATU LIMITED  
3      COUNTRY FOODS GISBORNE 1997  
4              NATIONAL COMMERCIAL  
0
100
200


In [8]:
# do some manual matching and then re enter the csv
# once the matching has been done there will be pairs of matching customers for example:
# cust No: 101, root No:102
# cust No: 102, root No:101
# these need to be matched to one root No (both either 101 or 102 in the example)
# below the number is decided by picking either the shortest customer name or sorted by customer ID
filename_matched = '~/Documents/Projects/16_01_18_/ParentInfo/TEST_customer_matching.csv'
df_manual_match = pandas.read_csv(filename_matched, encoding='latin-1')

# create new fields to store the matching
df_manual_match['Root Customer ID'] = ''
df_manual_match['Root Customer Name'] = ''
print(df_manual_match.columns.values)

for i, row in df_manual_match.iterrows():
    if row['Customer ID'] != row['Matched ID']:
        
        # test which has the smallest customer name
        cust1_len = len(row['Orig Cust Name'])
        cust2_len = len(row['Orig Cust Name2'])
        if cust1_len < cust2_len:
            df_manual_match['Root Customer ID'].loc[i] = row['Customer ID']
            df_manual_match['Root Customer Name'].loc[i] = row['Orig Cust Name']
        # if the lengths are equal sort ID either by number or alphabetically
        elif cust1_len == cust2_len:
            custlist = {int(row['Customer ID']), int(row['Matched ID'])}
            alpha_cust = sorted(custlist)[0]
            #print(custlist, alpha_cust, cust1_len, cust2_len, row['Score'])
            if row['Customer ID'] == alpha_cust:
                df_manual_match['Root Customer ID'].loc[i] = row['Customer ID']
                df_manual_match['Root Customer Name'].loc[i] = row['Orig Cust Name']
            else:
                df_manual_match['Root Customer ID'].loc[i] = row['Matched ID']
                df_manual_match['Root Customer Name'].loc[i] = row['Orig Cust Name']
        else:
            df_manual_match['Root Customer ID'].loc[i] = row['Matched ID']
            df_manual_match['Root Customer Name'].loc[i] = row['Orig Cust Name']
    else:
        df_manual_match['Root Customer ID'].loc[i] = row['Matched ID']
        df_manual_match['Root Customer Name'].loc[i] = row['Orig Cust Name']
        
        # should be doing the setting like this i think
        # df_manual_match.ix[i, 'Root Customer Name'] = row['Orig Cust Name']
    if i%100==0: print(i) # for progress update
    #if i>100: break
        
# do some filtering and renaming and output to csv
df_manual_match = df_manual_match[['Customer ID', 'Orig Cust Name', 'Root Customer ID', 'Root Customer Name']]
df_manual_match.rename(columns={'Orig Cust Name':'Customer Name'}, inplace=True)
df_manual_match.to_csv("TEST_final_matching.csv")
# check the csv output for mistakes

['Unnamed: 0' 'Orig Cust Name' 'Orig Cust Name2' 'Customer Name'
 'Customer Name2' 'Customer ID' 'Customer ID 2' 'Matched ID' 'Score'
 'Root Customer ID' 'Root Customer Name']
0
100
200


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
